In [1]:
import pickle
from collections import Counter

import numpy
from keras.callbacks import ModelCheckpoint
from kutilities.callbacks import MetricsCallback, WeightsCallback, \
    PlottingCallback
from kutilities.helpers.data_preparation import get_labels_to_categories_map, \
    get_class_weights2, onehot_to_categories
from sklearn.metrics import precision_score, accuracy_score, \
    mean_absolute_error
from sklearn.metrics import recall_score

from dataset.data_loader import SemEvalDataLoader
from models.nn_models import target_RNN
from utilities.data_loader import get_embeddings, Task4Loader, prepare_dataset

Using TensorFlow backend.


In [2]:
numpy.random.seed(1337)  # for reproducibility

# specify the word vectors file to use.
# for example, WC_CORPUS = "own.twitter" and WC_DIM = 300,
# correspond to the file "datastories.twitter.300d.txt"
WV_CORPUS = "datastories.twitter"
WV_DIM = 50

# Flag that sets the training mode.
# - if FINAL == False,  then the dataset will be split in {train, val, test}
# - if FINAL == True,   then the dataset will be split in {train, val}.
# Even for training the model for the final submission a small percentage
# of the labeled data will be kept for as a validation set for early stopping
FINAL = True

# If True, the SemEval gold labels will be used as the testing set in order to perform Post-mortem analysis
POST_MORTEM = True

text_max_length = 50
target_max_length = 6
TASK = "CE"  # Specify the Subtask. It is needed to correctly load the data

In [3]:
############################################################################
# PERSISTENCE
############################################################################
# if True save model checkpoints, as well as the corresponding word indices
# you HAVE tp set PERSIST = True, in order to be able to use the trained model later
PERSIST = False
best_model = lambda: "cp_model_task4_sub{}.hdf5".format(TASK)
best_model_word_indices = lambda: "cp_model_task4_sub{}_word_indices.pickle".format(
    TASK)

In [4]:
############################################################################
# LOAD DATA
############################################################################
embeddings, word_indices = get_embeddings(corpus=WV_CORPUS, dim=WV_DIM)

if TASK == "BD":
    loader = Task4Loader(word_indices,
                         text_lengths=(target_max_length, text_max_length),
                         subtask=TASK,
                         filter_classes={"positive", "negative"},
                         y_one_hot=False)
    classes = ['positive', 'negative']
else:
    loader = Task4Loader(word_indices,
                         text_lengths=(target_max_length, text_max_length),
                         subtask=TASK)
    classes = ["-2", "-1", "0", "1", "2"]

if PERSIST:
    pickle.dump(word_indices, open(best_model_word_indices(), 'wb'))

if FINAL:
    print("\n > running in FINAL mode!\n")
    training, testing = loader.load_final()
else:
    training, validation, testing = loader.load_train_val_test()

if POST_MORTEM:
    print("\n > running in Post-Mortem mode!\n")
    gold_data = SemEvalDataLoader().get_gold(task=TASK)
    gX = [obs[1] for obs in gold_data]
    gy = [obs[0] for obs in gold_data]
    gold = prepare_dataset(gX, gy, loader.pipeline, loader.y_one_hot)

    validation = testing
    testing = gold
    FINAL = False

Loaded 658125 word vectors.
Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...
Loading data...



PreProcessing...:   1%|          | 171/27388 [00:00<00:15, 1708.55it/s]

total observations: 30432
-------------------
training set stats
-------------------
Total: 30432
{'-1': '3381 (11.11%)', '-2': '296 (0.97%)', '0': '12885 (42.34%)', '1': '12854 (42.24%)', '2': '1016 (3.34%)'}
-------------------

 > running in FINAL mode!


Preparing training set...
Total: 27388
{'-1': '3043 (11.11%)', '-2': '267 (0.97%)', '0': '11596 (42.34%)', '1': '11568 (42.24%)', '2': '914 (3.34%)'}


PreProcessing...:   7%|▋         | 223/3044 [00:00<00:01, 2228.26it/s]


Preparing test set...
Total: 3044
{'-1': '338 (11.10%)', '-2': '29 (0.95%)', '0': '1289 (42.35%)', '1': '1286 (42.25%)', '2': '102 (3.35%)'}


PreProcessing...:   0%|          | 0/12284 [00:00<?, ?it/s]


 > running in Post-Mortem mode!


Parsing file: SemEval2017-task4-test.subtask-CE.english.txt done!
done!
Total: 12284
{'-1': '3509 (28.57%)', '-2': '177 (1.44%)', '0': '6146 (50.03%)', '1': '2322 (18.90%)', '2': '130 (1.06%)'}


PreProcessing...: 100%|██████████| 12284/12284 [00:06<00:00, 1765.08it/s]


In [5]:
############################################################################
# NN MODEL
############################################################################
print("Building NN Model...")
nn_model = target_RNN(embeddings,
                      tweet_max_length=text_max_length,
                      aspect_max_length=target_max_length,
                      noise=0.2,
                      activity_l2=0.001,
                      drop_text_rnn_U=0.2,
                      drop_text_input=0.3,
                      drop_text_rnn=0.3,
                      drop_target_rnn=0.2,
                      use_final=True,
                      bi=True,
                      final_size=64,
                      drop_final=0.5,
                      lr=0.001,
                      rnn_cells=64,
                      attention="context",
                      clipnorm=.1,
                      classes=len(classes))

print(nn_model.summary())

Building NN Model...
Instructions for updating:
Colocations handled automatically by placer.


/Users/williamscott/playground/baseline2/datastories-semeval2017-task4/models/nn_models.py:43: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(75, return_sequences=True, kernel_regularizer=<keras.reg..., recurrent_dropout=0.2, implementation=0)`
  W_regularizer=l2(l2_reg))
/anaconda3/envs/temp/lib/python3.6/site-packages/keras/layers/recurrent.py:2068: UserWarning: `implementation=0` has been deprecated, and now defaults to `implementation=1`.Please update your layer call.
  warnings.warn('`implementation=0` has been deprecated, '


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 50)       32906350    input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 6)            0                                            
__________________________________________________________________________________________________
gaussian_noise_1 (GaussianNoise (None, 50, 50)       0           embedding_1[0][0]           

/Users/williamscott/playground/baseline2/datastories-semeval2017-task4/models/nn_models.py:180: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  model = Model(input=[input_aspect, input_tweet], output=probabilities)


In [6]:
############################################################################
# CALLBACKS
############################################################################

# define metrics and class weights
if TASK == "BD":
    cat_to_class_mapping = {v: k for k, v in
                            get_labels_to_categories_map(classes).items()}
    metrics = {
        "accuracy": (lambda y_test, y_pred: accuracy_score(y_test, y_pred)),
        "recall": (lambda y_test, y_pred: recall_score(y_test, y_pred,
                                                       average='macro')),
        "precision": (lambda y_test, y_pred: precision_score(y_test, y_pred,
                                                             average='macro'))
    }
else:
    cat_to_class_mapping = {v: int(k) for k, v in
                            get_labels_to_categories_map(classes).items()}


    def macro_mae(y_test, y_pred):
        _y_test = [cat_to_class_mapping[y] for y in y_test]
        _y_pred = [cat_to_class_mapping[y] for y in y_pred]

        c = Counter(_y_pred)
        print(c)

        classes = set(_y_test)
        micro_m = {}
        for c in classes:
            class_sentences = [(t, p) for t, p in zip(_y_test, _y_pred) if
                               t == c]
            yt = [y[0] for y in class_sentences]
            yp = [y[1] for y in class_sentences]
            micro_m[c] = mean_absolute_error(yt, yp)
        # pprint.pprint(sorted(micro_m.items(), key=lambda x: x[1], reverse=True))
        return numpy.mean(list(micro_m.values()))


    metrics = {
        "macro_mae": macro_mae,
        "micro_mae": (
            lambda y_test, y_pred: mean_absolute_error(y_test, y_pred)),
    }

_datasets = {}
_datasets["1-train"] = (training[0], training[1]),
_datasets["2-val"] = (validation[0], validation[1]) if not FINAL else (
    testing[0], testing[1])
if not FINAL:
    _datasets["3-test"] = (testing[0], testing[1])

metrics_callback = MetricsCallback(datasets=_datasets, metrics=metrics)
weights = WeightsCallback(parameters=["W"], stats=["raster", "mean", "std"])

if TASK == "BD":
    plotting = PlottingCallback(grid_ranges=(0.75, 1), height=4,
                                benchmarks={"ρ": 0.797, "α": 0.87})
    checkpointer = ModelCheckpoint(filepath=best_model(), monitor='val.recall',
                                   mode="max", verbose=1, save_best_only=True)
else:
    plotting = PlottingCallback(grid_ranges=(0.4, 1.), height=4,
                                benchmarks={"MAE_M": 0.719, "MAE_m": 0.58})
    checkpointer = ModelCheckpoint(filepath=best_model(),
                                   monitor='val.macro_mae', mode="min",
                                   verbose=1, save_best_only=True)

_callbacks = []
_callbacks.append(metrics_callback)
_callbacks.append(plotting)
_callbacks.append(weights)

In [7]:
############################################################################
# APPLY CLASS WEIGHTS
############################################################################
if TASK == "BD":
    class_weights = get_class_weights2(training[1], smooth_factor=0)
else:
    class_weights = get_class_weights2(onehot_to_categories(training[1]),
                                       smooth_factor=0.1)

print("Class weights:",
      {cat_to_class_mapping[c]: w for c, w in class_weights.items()})

Class weights: {2: 6.151427469135801, 0: 1.0, 1: 1.0021999434300262, -1: 3.035168705087327, -2: 8.94125893733352}


In [ ]:
history = nn_model.fit(training[0], training[1],
                       validation_data=(
                           validation[0], validation[1]) if not FINAL else (
                           testing[0], testing[1]),
                       nb_epoch=50, batch_size=2, class_weight=class_weights,
                       callbacks=_callbacks)

Instructions for updating:
Use tf.cast instead.


/anaconda3/envs/temp/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 27388 samples, validate on 3044 samples
Epoch 1/50


In [ ]:
pickle.dump(history.history,
            open("hist_task4_sub{}.pickle".format(TASK), "wb"))